In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#from sklearn.feature_extraction.text import CountVectorizer  #DT does not take strings as input for the model fit step....
from IPython.display import Image  
#import pydotplus as pydot
from sklearn import tree
from os import system
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
def read_file(Filename):
    df=pd.read_csv(Filename)
    return df

In [3]:
def null_remove(df):
    df1=df.dropna(inplace=False)
    return df1

In [4]:
def target_datacoversion(df):
    df1 = df.copy()
    df1['Churn'] =df['Churn'].replace({"No":0, "Yes":1})
    return df1

In [5]:
def remove_low_cardinality(df):
    low_card_cols_1=df.columns[df.nunique()==1].tolist()
    cleansed_df =df.drop(columns=low_card_cols_1)
    return cleansed_df 

In [6]:
def datatype_con(df,col_list):
    for  i in col_list:
        df[i]=pd.to_numeric(df[i],errors='coerce')
    return df

In [7]:
def one_hot_encode(df):
    col_list = df.select_dtypes(include='object').columns.to_list()
    df_one_hot = pd.get_dummies(df, columns = col_list)
    return df_one_hot

In [8]:
def split_data(df_cleanse,Predict_col,split_size):
    y=df_cleanse[Predict_col] #target
    X=df_cleanse.drop(columns=Predict_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_size, random_state=10)
    return (X_train,X_test,y_train,y_test)

In [9]:
def model_fit(X_train,y_train,Modlename):
    if(Modlename=='XGB'):
        xgb_model = XGBClassifier()
        fitted_model=xgb_model.fit(X_train, y_train)
    elif(Modlename=='Logit'):
        logit = LogisticRegression()
        fitted_model=logit.fit(X_train, y_train)
    elif(Modlename=='KNN'):
        KNN = KNeighborsClassifier(n_neighbors= 5 , metric = 'euclidean' ) 
        fitted_model=KNN.fit(X_train, y_train)
    elif(Modlename=='SVM'):
        clf = svm.SVC(gamma=0.025, C=3)     
        fitted_model=clf.fit(X_train, y_train)
    elif(Modlename=='DTREE'):
        dTree = DecisionTreeClassifier(criterion = 'gini', random_state=1)  
        fitted_model=dTree.fit(X_train, y_train)  
    elif(Modlename=='RFCL'):
        rfcl = RandomForestClassifier(n_estimators = 50, random_state=1,max_features=12) 
        fitted_model=rfcl.fit(X_train, y_train) 
    return fitted_model

In [10]:
def model_score(Fit_model,X_test,y_test):         
    y_pred = Fit_model.predict(X_test)
    print(metrics.classification_report(y_test, y_pred))
    accuracy_score=metrics.accuracy_score(y_test,y_pred)*100
    return accuracy_score

In [11]:
def data_pre_procssing(df,Val_Target_var,col_list_datatype,ID_Column):
    #Drop the Feature which is quique for each Row 
    df.drop(ID_Column,axis=1,inplace=True)
    #Remove the Feature which is having Low Cardinality (1)
    df1=remove_low_cardinality(df)
    print("df1 Columns",df1.columns)
    #Convert Datatype Conversion 
    df2=datatype_con(df1,col_list_datatype)
    print("df2 Columns",df2.columns)
    #Remove the Null Components in the feature
    df3=null_remove(df2)
    print("df3 Columns",df3.columns)
    #Target Variable conversion to Decimal Values
    df4=target_datacoversion(df3)
    #One Hot conversion for Categorial Features
    df_final=one_hot_encode(df4)
    return df_final

In [12]:
def best_performing_model(accuracy_score):
    max_v =-99
    best_model=''
    for i in accuracy_score:
        if accuracy_score[i] > max_v:
            max_v =accuracy_score[i]
            best_model =i
    return best_model

In [13]:
def main():
    
    # Accept Data file form the user and analyse the features which needs to be processed
    val_fileName = input("Enter Value of CSV File Name to be modelled: ")
    df=read_file(val_fileName)
    print("Data Shape",df.shape)
    print("Data Info",df.info())
    
    
    print("Data Cleansing/Preparation Started")
    print("Analysis data and determine features which needs to be processed(Data type Conversioin,Unique Value)")
    Val_Target_var=input("Enter Target Column for Classification: ")
    #TelcomCustomer-Churn_2.csv
    col_datatype_conversion =input("Enter List of Columns to be Converted to Decimal")
    ID_Column =input("Enter the Column which is ID and which unique for each Row")
    col_list_datatype_col = col_datatype_conversion.split(',')  
    #Calling Data cleansing Function 
    df_cleansed=data_pre_procssing(df,Val_Target_var,col_list_datatype_col,ID_Column)
    print("Data Cleansing/Preparation Completed")
    
    #Split the Data set as training/Test data sets 
    print("Split data for Modelling started")
    val_split_size =float(input("Enter the Split size in Decimal"))
    X_train, X_test, y_train, y_test= split_data(df_cleansed,Val_Target_var,val_split_size)
    print("Split data for Modelling Completed")
    
    #List of Models to be trained & Scored 
    Model_list =['XGB','KNN','Logit','SVM','DTREE','RFCL']
    model_arr={}
    model_a_score={}
    for i in Model_list:
        model_arr[i]=model_fit(X_train,y_train,i)
        model_a_score[i]=model_score(model_arr[i],X_test,y_test)
    print(model_arr)
    print(model_a_score)
    best_model=best_performing_model(model_a_score)
    print("********************************************")
    print("Best Model Choosed based on Accuracy Score",best_model)
    print("********************************************")
    a=model_arr[best_model]
    # Storing the best Perfomance Model as pickle FIle 
    pickle.dump(a,open('models/Best_performing_model_'+best_model+'.p','wb'))

In [14]:

    main()

Enter Value of CSV File Name to be modelled: TelcomCustomer-Churn_2.csv
Data Shape (7043, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   OnlineBackup      7043 non-null   object 
 2   DeviceProtection  7043 non-null   object 
 3   TechSupport       7043 non-null   object 
 4   StreamingTV       7043 non-null   object 
 5   StreamingMovies   7043 non-null   object 
 6   Contract          7043 non-null   object 
 7   PaperlessBilling  7043 non-null   object 
 8   PaymentMethod     7043 non-null   object 
 9   MonthlyCharges    7043 non-null   float64
 10  TotalCharges      7043 non-null   object 
 11  Churn             7043 non-null   object 
dtypes: float64(1), object(11)
memory usage: 660.4+ KB
Data Info None
Data Cleansing/Preparation Started
Analysis data and determine features

/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:43:44] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1046
           1       0.62      0.56      0.59       361

    accuracy                           0.80      1407
   macro avg       0.74      0.72      0.73      1407
weighted avg       0.79      0.80      0.80      1407

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      1046
           1       0.54      0.47      0.50       361

    accuracy                           0.76      1407
   macro avg       0.68      0.66      0.67      1407
weighted avg       0.75      0.76      0.75      1407



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1046
           1       0.63      0.54      0.58       361

    accuracy                           0.80      1407
   macro avg       0.74      0.71      0.72      1407
weighted avg       0.79      0.80      0.79      1407

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      1046
           1       0.50      0.44      0.47       361

    accuracy                           0.74      1407
   macro avg       0.66      0.64      0.65      1407
weighted avg       0.73      0.74      0.74      1407

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1046
           1       0.51      0.53      0.52       361

    accuracy                           0.75      1407
   macro avg       0.67      0.68      0.67      1407
weighted avg       0.75      0.75      0.75      1407

              preci